In [1]:
import os
import shutil
import re

In [73]:
#
# DO NOT CHANGE THIS CODE BLOCK
#

def copy_linked_files(parent_directory, root_markdown_file, new_location, ignore_files, ignore_prefixes, PREFIX_LENGTH=10):
    # Create the new location if it doesn't already exist
    if not os.path.exists(new_location):
        os.makedirs(new_location)
    searched_files = set()
    files_to_copy = []
    # Create a queue to hold the markdown files to be searched
    queue = []
    for dirpath, dirnames, files in os.walk(parent_directory):
        for file in files:
            if file == root_markdown_file:
                root_markdown_path = os.path.join(dirpath, file)
                queue.append(root_markdown_path)
                break
                
    print("Searching for linked files...")
    while queue:
        # Get the next markdown file from the queue
        current_file = queue.pop(0)
        
#         print(current_file)
        if current_file in searched_files:
            continue
            
        searched_files.add(current_file)
        # Read the markdown file
        with open(current_file, 'r') as f:
            markdown_text = f.read()

        # Use regular expression to extract the filenames inside [[ and ]]
        filenames = re.findall(r'\[\[(.+?)([|#^].*)?\]\]', markdown_text)

        # Iterate through the filenames
        for filename in filenames:
            
            # get prefix
            ignore_file = False
            location_index = markdown_text.find(filename[0])
            prefix = markdown_text[location_index-PREFIX_LENGTH:location_index]
            for ig_prefix in ignore_prefixes:
                if (ig_prefix == prefix):
                    ignore_file = True
                    break
            if(ignore_file):
                continue
            
            for dirpath, dirnames, files in os.walk(parent_directory):
                for file in files:
                    if (file == filename[0]) or (file == os.path.splitext(filename[0])[0] + '.md'):
                        full_path = os.path.join(dirpath, file)
                        
                        for ignored_file in ignore_files:
                            if (ignored_file == os.path.splitext(filename[0])[0] + '.md'):
                                searched_files.add(full_path)
                        
                        if os.path.isfile(full_path) and (full_path not in searched_files):
                            if full_path.endswith('.md'):
                                files_to_copy.append(full_path)
                                queue.append(full_path)
                                # check if there is a pdf file in the same directory with the same name
                                pdf_path = os.path.splitext(full_path)[0] + '.pdf'
                                if os.path.isfile(pdf_path):
                                    files_to_copy.append(pdf_path)
                            else:
                                files_to_copy.append(full_path)
            print('\r', end='', flush=True)
            print("Searched {} files, {} files in queue, {} files to copy.".format(len(searched_files), len(queue), len(files_to_copy)), end='')
    print("\nFound {} files, copying them to {}...".format(len(files_to_copy), new_location))
    for file_path in files_to_copy:
        # Construct the relative path of the file in the new location
        relative_path = os.path.relpath(file_path, parent_directory)
        new_path = os.path.join(new_location, relative_path)
        # Create the directory if it doesn't already exist
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        # Copy the file
        shutil.copy2(file_path, new_path)
    print("Finished copying files.")

In [74]:
#
# CHANGE THESE VARIABLES TO MATCH YOUR LIKINGS
#
# @param parent_directory     absolute path to the vault directory, only files from this directory are copied
# @param new_location         absolute path to the newly created sub-vault directory
# @param root_markdown        MD filename of the root file to search for linked files
# @param ignore_files         MD filenames that are ignored and not searched nor copied
# @param ignore_prefixes      Trailing 10 (by default) characters before a filelink, that if present, do not
#                             search nor copy the following linked file
#

parent_directory = '/home/michael/Documents/Michaels Vault'
root_markdown    = 'ct.md'
new_location     = '/home/michael/Documents/Vault-CT'

ignore_files     = ['semester-09.md', 'semester-08.md']
ignore_prefixes  = ['ture:** [[', 'file:** [[']

In [75]:
copy_linked_files(parent_directory, root_markdown, new_location, ignore_files, ignore_prefixes)

Searching for linked files...
Searched 196 files, 0 files in queue, 1089 files to copy...
Found 1089 files, copying them to /home/michael/Documents/Vault-CT...
Finished copying files.
